In [106]:
!pip install yfinance


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
import yfinance as yf
import pandas as pd
from google.colab import files

In [108]:
ticker_symbol = "META"  # Replace with the desired ticker symbol


In [109]:
stock_data = yf.download(ticker_symbol, interval="1d")
stock_data.reset_index(inplace=True)  # Reset the index to have the "Date" column



[*********************100%***********************]  1 of 1 completed


In [110]:
stock_data["Returns"] = stock_data["Close"].pct_change()
stock_data["MovingAverage_50"] = stock_data["Close"].rolling(window=50).mean()
stock_data["MovingAverage_200"] = stock_data["Close"].rolling(window=200).mean()
stock_data["VolumeChange"] = stock_data["Volume"].pct_change()
stock_data["VolumeMovingAverage_50"] = stock_data["Volume"].rolling(window=50).mean()
stock_data["VolumeMovingAverage_200"] = stock_data["Volume"].rolling(window=200).mean()
risk_free_rate = 0.02
stock_data["MarketReturns"] = stock_data["Returns"].mean()
stock_data["CAPM"] = risk_free_rate + stock_data["MarketReturns"] * (stock_data["Returns"] - risk_free_rate)
stock_data["HighLowRange"] = stock_data["High"] - stock_data["Low"]
stock_data["HighCloseRange"] = stock_data["High"] - stock_data["Close"].shift()
stock_data["LowCloseRange"] = stock_data["Close"].shift() - stock_data["Low"]
stock_data["Volatility"] = stock_data[["HighLowRange", "HighCloseRange", "LowCloseRange"]].mean(axis=1)



In [111]:
window = 14
delta = stock_data["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
average_gain = gain.rolling(window=window).mean()
average_loss = loss.rolling(window=window).mean()
relative_strength = average_gain / average_loss
stock_data["RSI"] = 100 - (100 / (1 + relative_strength))


In [112]:
# Momentum factors
window = 10
stock_data["Momentum"] = stock_data["Close"] - stock_data["Close"].shift(window)

# Volume-related features
stock_data["VolumeChange"] = stock_data["Volume"].pct_change()
stock_data["VolumeMovingAverage_10"] = stock_data["Volume"].rolling(window=10).mean()

# Price/return lags
lags = [1, 3, 5]  # Define the desired lag periods
for lag in lags:
    stock_data[f"PriceLag_{lag}"] = stock_data["Close"].shift(lag)
    stock_data[f"ReturnLag_{lag}"] = stock_data["Returns"].shift(lag)



In [113]:
stock_data.to_csv("meta_stock_features.csv")


In [114]:
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,MovingAverage_50,MovingAverage_200,...,Volatility,RSI,Momentum,VolumeMovingAverage_10,PriceLag_1,ReturnLag_1,PriceLag_3,ReturnLag_3,PriceLag_5,ReturnLag_5
0,2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400,NaN,NaN,NaN,...,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700,-0.109861,NaN,NaN,...,2.440000,NaN,NaN,NaN,38.230000,NaN,NaN,NaN,NaN,NaN
2,2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600,-0.089039,NaN,NaN,...,1.766666,NaN,NaN,NaN,34.029999,-0.109861,NaN,NaN,NaN,NaN
3,2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000,0.032258,NaN,NaN,...,0.760000,NaN,NaN,NaN,31.000000,-0.089039,38.230000,NaN,NaN,NaN
4,2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200,0.032187,NaN,NaN,...,0.959999,NaN,NaN,NaN,32.000000,0.032258,34.029999,-0.109861,NaN,NaN


In [115]:
fama_french_data = pd.read_csv('/F-F_Research_Data_5_Factors_2x3_daily.CSV', skiprows=3)


In [116]:
fama_french_data.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF
0,19630701,-0.67,0.02,-0.35,0.03,0.13,0.012
1,19630702,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,19630703,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,19630705,0.40,0.09,-0.28,0.07,-0.30,0.012
4,19630708,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [117]:
fama_french_data["Date"] = pd.to_datetime(fama_french_data["Unnamed: 0"], format="%Y%m%d")
fama_french_data.drop("Unnamed: 0", axis=1, inplace=True)
print(fama_french_data.head())


   Mkt-RF   SMB   HML   RMW   CMA     RF       Date
0   -0.67  0.02 -0.35  0.03  0.13  0.012 1963-07-01
1    0.79 -0.28  0.28 -0.08 -0.21  0.012 1963-07-02
2    0.63 -0.18 -0.10  0.13 -0.25  0.012 1963-07-03
3    0.40  0.09 -0.28  0.07 -0.30  0.012 1963-07-05
4   -0.63  0.07 -0.20 -0.27  0.06  0.012 1963-07-08


In [118]:
merged_data = pd.merge(stock_data, fama_french_data, on="Date", how="inner")


In [119]:
merged_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,MovingAverage_50,MovingAverage_200,...,PriceLag_3,ReturnLag_3,PriceLag_5,ReturnLag_5,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.80,-0.15,0.07,0.16,0.38,0.0
1,2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700,-0.109861,NaN,NaN,...,NaN,NaN,NaN,NaN,1.69,0.55,-1.10,-0.29,-0.63,0.0
2,2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600,-0.089039,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,-0.85,0.35,-0.11,-0.01,0.0
3,2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000,0.032258,NaN,NaN,...,38.230000,NaN,NaN,NaN,0.26,0.50,-0.23,-0.04,-0.07,0.0
4,2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200,0.032187,NaN,NaN,...,34.029999,-0.109861,NaN,NaN,0.16,0.02,0.18,0.36,0.54,0.0


In [120]:
ADS_Index_Most_Current_Vintage = pd.read_csv('/ADS_Index_Most_Current_Vintage.csv')
ADS_Index_Most_Current_Vintage["Date"] = pd.to_datetime(ADS_Index_Most_Current_Vintage["Unnamed: 0"], format="%Y:%m:%d")
ADS_Index_Most_Current_Vintage.drop("Unnamed: 0", axis=1, inplace=True)

In [121]:
merged_data2 = pd.merge(merged_data, ADS_Index_Most_Current_Vintage, on="Date", how="inner")
merged_data2.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,MovingAverage_50,MovingAverage_200,...,ReturnLag_3,PriceLag_5,ReturnLag_5,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index
0,2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.80,-0.15,0.07,0.16,0.38,0.0,-0.127285
1,2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700,-0.109861,NaN,NaN,...,NaN,NaN,NaN,1.69,0.55,-1.10,-0.29,-0.63,0.0,-0.163240
2,2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600,-0.089039,NaN,NaN,...,NaN,NaN,NaN,0.00,-0.85,0.35,-0.11,-0.01,0.0,-0.174997
3,2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000,0.032258,NaN,NaN,...,NaN,NaN,NaN,0.26,0.50,-0.23,-0.04,-0.07,0.0,-0.186586
4,2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200,0.032187,NaN,NaN,...,-0.109861,NaN,NaN,0.16,0.02,0.18,0.36,0.54,0.0,-0.198012


In [122]:
UNRATE = pd.read_csv('/UNRATE.csv')

In [123]:


# Read the monthly data from the CSV file
df = pd.read_csv('/UNRATE.csv')

# Convert the "DATE" column to datetime type
df['Date'] = pd.to_datetime(df['DATE'])

# Set the "DATE" column as the DataFrame index
df.set_index('Date', inplace=True)

# Define the start and end dates
start_date = df.index.min()
end_date = df.index.max()

# Create a daily date range
daily_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Reindex the DataFrame with the daily dates
df_daily = df.reindex(daily_dates)

# Forward fill the missing values with the last observed value
df_daily['UNRATE'] = df_daily['UNRATE'].ffill()

# Reset the index to include the "DATE" column
df_daily.reset_index(inplace=True)

# Rename the columns if needed
df_daily.rename(columns={'index': 'Date'}, inplace=True)
# Drop the "DATE" column from merged_data3
df_daily.drop("DATE", axis=1, inplace=True)

# Print the resulting DataFrame
print(df_daily)


           Date  UNRATE
0    2012-05-01     8.2
1    2012-05-02     8.2
2    2012-05-03     8.2
3    2012-05-04     8.2
4    2012-05-05     8.2
...         ...     ...
4013 2023-04-27     3.4
4014 2023-04-28     3.4
4015 2023-04-29     3.4
4016 2023-04-30     3.4
4017 2023-05-01     3.7

[4018 rows x 2 columns]


In [124]:
merged_data3 = pd.merge(merged_data2, df_daily, on="Date", how="inner")
merged_data3.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,MovingAverage_50,MovingAverage_200,...,PriceLag_5,ReturnLag_5,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index,UNRATE
0,2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400,NaN,NaN,NaN,...,NaN,NaN,-0.80,-0.15,0.07,0.16,0.38,0.0,-0.127285,8.2
1,2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700,-0.109861,NaN,NaN,...,NaN,NaN,1.69,0.55,-1.10,-0.29,-0.63,0.0,-0.163240,8.2
2,2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600,-0.089039,NaN,NaN,...,NaN,NaN,0.00,-0.85,0.35,-0.11,-0.01,0.0,-0.174997,8.2
3,2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000,0.032258,NaN,NaN,...,NaN,NaN,0.26,0.50,-0.23,-0.04,-0.07,0.0,-0.186586,8.2
4,2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200,0.032187,NaN,NaN,...,NaN,NaN,0.16,0.02,0.18,0.36,0.54,0.0,-0.198012,8.2


In [125]:


# Define the stock tickers
tickers = ["TCEHY",  "SNAP"]

# Fetch lifetime stock data from yfinance
Opp_stock_data = yf.download(tickers, start="1900-01-01")

# Extract the "Close" prices
close_prices = Opp_stock_data["Close"]

# Save the data to a CSV file
close_prices.to_csv("Opp_stock_data.csv")


[*********************100%***********************]  2 of 2 completed


In [126]:
merged_data4 = pd.merge(merged_data3, Opp_stock_data, on="Date", how="left")


<ipython-input-126-ceb5463ace0f>:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  merged_data4 = pd.merge(merged_data3, Opp_stock_data, on="Date", how="left")


In [127]:

merged_data4.to_csv("Meta_Stock_Analysis_Data.csv")
files.download("Meta_Stock_Analysis_Data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>